In [2]:
import pandas as pd

from sqlalchemy import (create_engine, 
                        inspect, 
                        text, 
                        select, 
                        MetaData, 
                        Table, 
                        and_,
                        or_,
                        desc,
                        asc,
                        func,
                        )


engine = create_engine("sqlite+pysqlite:////workspaces/ERN-sessions/SQL in python/chinook.db")

gravity_engine = create_engine("sqlite+pysqlite://///workspaces/ERN-sessions/SQL in python/gravity.db")

In [4]:
metadata = MetaData() 

tracks = Table('tracks', metadata, autoload_with=engine)
print(repr(tracks)) # repr() function lets us vierw the details of our table, like .info() for a df

with engine.connect() as con:
    stmt = select(tracks).order_by(tracks.columns.Composer)
    result_proxy = con.execute(stmt)
    results = result_proxy.fetchmany(size=10) 
print(results)

df=pd.DataFrame(results)
print(df.info())
print(df)

Table('tracks', MetaData(), Column('TrackId', INTEGER(), table=<tracks>, primary_key=True, nullable=False), Column('Name', NVARCHAR(length=200), table=<tracks>, nullable=False), Column('AlbumId', INTEGER(), ForeignKey('albums.AlbumId'), table=<tracks>), Column('MediaTypeId', INTEGER(), ForeignKey('media_types.MediaTypeId'), table=<tracks>, nullable=False), Column('GenreId', INTEGER(), ForeignKey('genres.GenreId'), table=<tracks>), Column('Composer', NVARCHAR(length=220), table=<tracks>), Column('Milliseconds', INTEGER(), table=<tracks>, nullable=False), Column('Bytes', INTEGER(), table=<tracks>), Column('UnitPrice', NUMERIC(precision=10, scale=2), table=<tracks>, nullable=False), schema=None)
[(2, 'Balls to the Wall', 2, 2, 1, None, 342562, 5510424, Decimal('0.99')), (63, 'Desafinado', 8, 1, 2, None, 185338, 5990473, Decimal('0.99')), (64, 'Garota De Ipanema', 8, 1, 2, None, 285048, 9348428, Decimal('0.99')), (65, 'Samba De Uma Nota Só (One Note Samba)', 8, 1, 2, None, 137273, 4535401,

In [6]:
gravity_metadata = MetaData() 

books = Table('book', gravity_metadata, autoload_with=gravity_engine)
print(repr(books))

with gravity_engine.connect() as con:
    stmt = select(books).order_by(books.columns.title)
    result_proxy = con.execute(stmt)
    results = result_proxy.fetchmany(size=10) 
print(results)

df=pd.DataFrame(results)
print(df.info())
print(df)

Table('book', MetaData(), Column('book_id', INTEGER(), table=<book>, primary_key=True), Column('title', TEXT(), table=<book>), Column('isbn13', TEXT(), table=<book>), Column('language_id', INTEGER(), ForeignKey('book_language.language_id'), table=<book>), Column('num_pages', INTEGER(), table=<book>), Column('publication_date', DATE(), table=<book>), Column('publisher_id', INTEGER(), ForeignKey('publisher.publisher_id'), table=<book>), schema=None)
[(9940, '$30 Film School: How to Write  Direct  Produce  Shoot  Edit  Distribute  Tour With  and Sell Your Own No-Budget Digital Movie', '9781592000678', 1, 528, datetime.date(2003, 5, 13), 368), (7104, '1 000 Places to See Before You Die', '9780761104841', 1, 992, datetime.date(2003, 5, 22), 2224), (3938, '10 lb Penalty', '9780425197455', 1, 320, datetime.date(2004, 8, 3), 749), (3295, '100 Great Fantasy Short Short Stories', '9780380699179', 1, 395, datetime.date(1985, 8, 1), 160), (2068, '100 Love Sonnets', '9780292760288', 1, 232, datetim